# 🔄 LangGraph Iterative Workflows - Practical Examples

Is notebook mein hum **iterative workflows** ke practical examples dekhenge. Iterative workflows mein loops aur cycles hote hain jo progressive improvement, retries, aur self-correction ke liye use hote hain.

## Topics Covered:
1. Simple Counter Loop
2. Quality-Based Iteration
3. Retry Logic with Backoff
4. LLM-Based Content Refinement
5. Self-Correcting Review System
6. Data Processing with Validation Loop

**Reference**: Detailed explanations ke liye `ITERATIVE_WORKFLOW_GUIDE.md` dekhen.


## 📦 Setup & Imports

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from operator import add
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import time
import random

# Load environment variables
load_dotenv()

print("✅ All imports successful!")


## 1️⃣ Example 1: Simple Counter Loop

**Use Case**: Ek basic counter jo specified limit tak iterate karta hai.

**Key Concepts**:
- Loop counter
- Max iterations limit
- State accumulation with `Annotated[list, add]`
- Conditional exit


In [ ]:
# State definition
class CounterState(TypedDict):
    counter: int
    max_count: int
    history: Annotated[list[int], add]  # Accumulates across iterations

# Node: Increment counter
def increment(state: CounterState) -> CounterState:
    """Increment counter and record in history"""
    new_count = state["counter"] + 1
    print(f"🔄 Iteration {new_count}")
    return {
        "counter": new_count,
        "history": [new_count]  # Will be accumulated
    }

# Router: Check if we should continue
def should_continue(state: CounterState) -> Literal["continue", "end"]:
    """Check if we should continue looping"""
    if state["counter"] >= state["max_count"]:
        print("✅ Max count reached!")
        return "end"
    return "continue"

# Build graph
graph = StateGraph(CounterState)
graph.add_node("increment", increment)

graph.add_edge(START, "increment")
graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",  # Loop back to itself!
        "end": END
    }
)

counter_workflow = graph.compile()

# Test
print("\n" + "="*50)
print("Testing Counter Loop")
print("="*50 + "\n")

result = counter_workflow.invoke({
    "counter": 0,
    "max_count": 5,
    "history": []
})

print(f"\n📊 Final Result:")
print(f"   Counter: {result['counter']}")
print(f"   History: {result['history']}")


## 2️⃣ Example 2: Quality-Based Iteration

**Use Case**: Text ko iteratively improve karo jab tak quality threshold achieve na ho.

**Key Concepts**:
- Quality scoring
- Progressive improvement
- Multiple exit conditions (quality OR max iterations)
- Feedback loop


In [ ]:
# State definition
class QualityState(TypedDict):
    text: str
    quality_score: float
    iteration: int
    max_iterations: int
    improvements: Annotated[list[str], add]

# Node: Generate or improve text
def generate_text(state: QualityState) -> QualityState:
    """Generate or improve text"""
    if state["iteration"] == 0:
        text = "LangGraph is a framework."
        improvement = "Initial generation"
    else:
        # Progressively improve
        text = state["text"] + " It helps build stateful workflows."
        improvement = f"Added detail (iteration {state['iteration'] + 1})"
    
    print(f"🔄 Iteration {state['iteration'] + 1}: {improvement}")
    
    return {
        "text": text,
        "iteration": state["iteration"] + 1,
        "improvements": [improvement]
    }

# Node: Evaluate quality
def evaluate_quality(state: QualityState) -> QualityState:
    """Evaluate text quality"""
    # Simple quality metric: word count
    word_count = len(state["text"].split())
    quality = min(word_count / 20.0, 1.0)  # Target: 20 words
    
    print(f"   📊 Quality Score: {quality:.2f} (Words: {word_count})")
    
    return {"quality_score": quality}

# Router: Check if quality is good enough
def check_quality(state: QualityState) -> Literal["improve", "done"]:
    """Decide if we need more iterations"""
    if state["iteration"] >= state["max_iterations"]:
        print("⏱️  Max iterations reached")
        return "done"
    
    if state["quality_score"] >= 0.8:
        print("✅ Quality threshold achieved!")
        return "done"
    
    return "improve"

# Build graph
graph = StateGraph(QualityState)
graph.add_node("generate", generate_text)
graph.add_node("evaluate", evaluate_quality)

graph.add_edge(START, "generate")
graph.add_edge("generate", "evaluate")
graph.add_conditional_edges(
    "evaluate",
    check_quality,
    {
        "improve": "generate",  # Loop back!
        "done": END
    }
)

quality_workflow = graph.compile()

# Test
print("\n" + "="*50)
print("Testing Quality-Based Iteration")
print("="*50 + "\n")

result = quality_workflow.invoke({
    "text": "",
    "quality_score": 0.0,
    "iteration": 0,
    "max_iterations": 10,
    "improvements": []
})

print(f"\n📊 Final Result:")
print(f"   Quality: {result['quality_score']:.2f}")
print(f"   Iterations: {result['iteration']}")
print(f"   Text: {result['text']}")
print(f"   Improvements: {result['improvements']}")


## 3️⃣ Example 3: Retry Logic with Exponential Backoff

**Use Case**: Failed operations ko retry karna with increasing delays.

**Key Concepts**:
- Retry mechanism
- Exponential backoff (2^attempt seconds)
- Success/failure tracking
- Multiple exit paths (success/failed/retry)


In [ ]:
# State definition
class RetryState(TypedDict):
    attempt: int
    max_attempts: int
    success: bool
    result: str
    errors: Annotated[list[str], add]

# Node: Try operation
def try_operation(state: RetryState) -> RetryState:
    """Attempt the operation (simulated with random success)"""
    attempt_num = state["attempt"] + 1
    print(f"🔄 Attempt {attempt_num}...")
    
    # Simulate operation (60% failure rate for demo)
    success = random.random() > 0.6
    
    if success:
        print(f"   ✅ Success!")
        return {
            "attempt": attempt_num,
            "success": True,
            "result": "Operation succeeded!"
        }
    else:
        print(f"   ❌ Failed - will retry with backoff")
        return {
            "attempt": attempt_num,
            "success": False,
            "errors": [f"Attempt {attempt_num} failed"]
        }

# Node: Wait before retry (exponential backoff)
def wait_backoff(state: RetryState) -> RetryState:
    """Wait before retrying (exponential backoff)"""
    backoff = min(2 ** (state["attempt"] - 1), 8)  # Cap at 8 seconds
    print(f"   ⏳ Waiting {backoff} seconds before retry...")
    time.sleep(backoff)
    return {}

# Router: Decide whether to retry
def should_retry(state: RetryState) -> Literal["retry", "success", "failed"]:
    """Decide whether to retry"""
    if state["success"]:
        return "success"
    elif state["attempt"] >= state["max_attempts"]:
        print(f"   ⚠️  Max attempts ({state['max_attempts']}) reached")
        return "failed"
    else:
        return "retry"

# Build graph
graph = StateGraph(RetryState)
graph.add_node("try", try_operation)
graph.add_node("wait", wait_backoff)

graph.add_edge(START, "try")
graph.add_conditional_edges(
    "try",
    should_retry,
    {
        "retry": "wait",
        "success": END,
        "failed": END
    }
)
graph.add_edge("wait", "try")  # After waiting, try again

retry_workflow = graph.compile()

# Test
print("\n" + "="*50)
print("Testing Retry Logic with Exponential Backoff")
print("="*50 + "\n")

result = retry_workflow.invoke({
    "attempt": 0,
    "max_attempts": 5,
    "success": False,
    "result": "",
    "errors": []
})

print(f"\n📊 Final Result:")
print(f"   Success: {result['success']}")
print(f"   Total Attempts: {result['attempt']}")
print(f"   Result: {result['result'] if result['result'] else 'Failed after max attempts'}")
print(f"   Errors: {result['errors']}")
